# Discovery of Frequent Item-sets and Association Rules

Frequent item-set and associations rules are methods used in data mining in order to mining transaction data. A transaction data is a data where each rows represent the set of all items related to a buyer, user etc.  

Given the follwing dataset :

    [[1,2,3],
    [1,4],
    [4,5],
    [1,2,4],
    [1,2,6,4,3],
    [2,6,3],
    [2,3,6]]
We need to find the all the frequent item-sets in data. And find association rules such as (1,2) -> 3, we means that there is a strong correlation that people who buy items 1 and 2 are more likely to buy item 3. 


# Python implementation


**Support of an item-set : **
The support of an item-set X is the number of times the item-set occurred in the transaction data : X.count

\begin{equation*}
Support(X) = X.count
\end{equation*}

**Support of a rule : **
However the support of a rule $X \rightarrow Y$ is the number of times X and Y occurred together by the length of the transaction data. 

\begin{equation*}
support(X \rightarrow Y) = \frac{(X \cup Y).count}{N}
\end{equation*}
where N = number of transactions. 

**Confidence of a rule :**

The confidence of a rule $X \rightarrow Y$ is the number of times X and Y occurred together by the support of X 
\begin{equation*}
confidence(X \rightarrow Y) = \frac{(X \cup Y).count}{X.count}
\end{equation*}

Confidence determines the predictability and the reliability of a rule. 

In [3]:
import numpy as np
import itertools 

def loadTestDataSetA():
    return [[1,2,3],[1,4],[4,5],[1,2,4],[1,2,6,4,3],[2,6,3],[2,3,6]]

def findsubsets(s, n): 
    return [set(i) for i in itertools.combinations(s, n)] 

def init_pass(data) :
    unique_items = []
    
    for transaction in data :
        for item in transaction :
            if {item} not in  unique_items :
                unique_items.append({item})

    return unique_items

def scanData(data, C1, minSupport = 0.4) :
    
    map = {}
    unique_items = set()
    for transaction in data :
        for candidate in C1 :
            if candidate.issubset(transaction) :
                if len(candidate) < 2 :
                    cand = tuple(candidate)[0]
                    if cand in map :
                        map[cand]  += 1
                    else :
                        map[cand] = 1      
                else :
                    
                    if tuple(candidate) in map :
                        map[tuple(candidate)]  += 1
                    else :
                        map[tuple(candidate)] = 1
                       
          
    map_supp = {k: v / len(data)  for k, v in map.items() if v / len(data) >= minSupport}
    unique_items = list(map_supp.keys())

    return unique_items, map, map_supp


Let's load the dataset :


In [5]:
data = loadTestDataSetA()
print(data)

[[1, 2, 3], [1, 4], [4, 5], [1, 2, 4], [1, 2, 6, 4, 3], [2, 6, 3], [2, 3, 6]]


We need to find the uniques item in transaction data, and calculate the frequent item-set for minsupp = 0.5

In [8]:
C1 = init_pass(data)
minSupp = 0.5
unique_items, maps, map_supp = scanData(data, C1, minSupp)
print("Unique items : ")
print(unique_items)
print("Frequent item-set : ")
print(map_supp)


Unique items : 
[1, 2, 3, 4]
Frequent item-set : 
{1: 0.5714285714285714, 2: 0.7142857142857143, 3: 0.5714285714285714, 4: 0.5714285714285714}


We see that the support for item 1 is around 57% 

**Frequent item-set: **

There is different level of frequent item-set :

1. 1-level frequent item-set
This is single item-sets that appears frequently in the transaction data with $support \geq minsup$ .
2. 2-level frequent item-set 
This is couple item-sets that appears frequently in the transaction data with $support \geq minsup$ . 
2. N-level frequent item-set
And so on...


** Downward Closure Property : **

*If an item-set has minimum support (or its support \textbf{sup} is larger than \textbf{minsup}), then its every non-empty subset also has minimum support.*


The property states clearly that if we have a N-level frequent item-sets, then its every non-empty subset are frequent item-sets level N-1.

### The apriori algorithm :

The Apriori algorithm is a bottom-up algorithm with begins by generating 1-level frequents item-sets and then used that to generate 2-level frequents item-sets and so on until there no new way to generate level N+1. 

The Apriori algorithm is implemented in two phases. First we generate the 1-level frequent item-sets, then we iterate over and over until we find all the levels item-sets. 

In general, We first generate (n)-level frequent item-set, and used that to generate candidates for (n+1)-level frequent item-set, by concatenating each element in the (n)-level frequent item-sets such as each subsets of each items the (n+1)-level frequent item-sets candidates are subset of the (n)-level frequent item-set. 


In [9]:
def candidate_generation(Fk, k) :
    
    length_k = len(Fk)
    Ck = []
    
    element = Fk[0]
    if isinstance(element, int) :
        Fk = [ [x] for x in Fk]
    else :
        Fk = [ list(x) for x in Fk]
        
    for i in range(length_k) :
        for j in range(i+1, length_k) :
            L1 = list(Fk[i])
            L2 = list(Fk[j])
            L1.sort()
            L2.sort()
            if (L1[:-1] == L2[:-1]) and (L1[-1] != L2[-1]) :
                candidate = set(L1) | set(L2)
                subsets =  findsubsets(candidate, k-1)
                ToAdd = True
                for subset in subsets :
                    if list(subset) not in Fk :
                        ToAdd = False 
                if ToAdd :
                    Ck.append(candidate)
    return Ck

def apriori_algorithm(dataset, minSupp) :
    
    C1 = init_pass(dataset)
    unique_items, maps, map_supp = scanData(dataset, C1, minSupp)
    #print("map : ", map)
    A = []
    B = []
    C = [] 
    A.append(unique_items)
    C.append(map_supp)
    k = 2
    while(unique_items) :
        Ck = candidate_generation(unique_items, k)
        unique_items, maper, map_supp = scanData(dataset, Ck, minSupp)
        maps.update(maper)
        A.append(unique_items)
        C.append(map_supp)
        k += 1
    
    return A, maps, C

In [10]:
unique_items, maps,  map_supp = apriori_algorithm(data, minSupp) 
print("Frequent unique_items: ")
print(unique_items)
print("map_supp : ")
print(map_supp)

Frequent unique_items: 
[[1, 2, 3, 4], [(2, 3)], []]
map_supp : 
[{1: 0.5714285714285714, 2: 0.7142857142857143, 3: 0.5714285714285714, 4: 0.5714285714285714}, {(2, 3): 0.5714285714285714}, {}]


### Association rules generation

During the running of the apriori-alorithm we record the support each frequent itemset and the rules that will be generated will emane from the frequent itemsets. 

We generate association rules by looping over each n in $[1,..,N]$ such as :
for each item-set $\mathcal{T}$ in n-level frequent item-set ($n > 1$ and $n \in [1,..,N]$ ), we generate each of its subset $\beta$, then we calculate, the confidence of rule $(\mathcal{T} - \beta) \rightarrow \beta $ :
\begin{equation*}
confidence((\mathcal{T} - \beta) \rightarrow \beta) = \frac{((\mathcal{T} - \beta) \rightarrow \beta).count}{(\mathcal{T} - \beta).count}
\end{equation*}

In [11]:
def generate_rules(dataset,minSupp = 0.5, conf = 0.7 ) :
    
    uniques, map, map_support = apriori_algorithm(dataset, minSupp) 
    rules = []
    for cnt, f in enumerate(uniques) :
        if cnt >= 1 : 
            for itemset in f :
                length_f = len(itemset)
                for i in range(1,length_f) :
                    subsets = findsubsets(itemset, i)
                    for beta in subsets :
                        f_b = set(itemset) - beta
                        
                        confidence = map[itemset] 
                        if len(f_b) <= 1 :
                            confidence = confidence * 1.0 / map[list(f_b)[0]] 
                        else :
                            confidence = confidence * 1.0 / map[tuple(f_b)] 
                            
                        if confidence >= conf :
                            rules.append((f_b, beta))
       
    return rules

In [16]:
conf = 0.9
minSupp = 0.4
rules = generate_rules(data,minSupp,conf)
print("The rules !!!! ")
print(rules)

The rules !!!! 
[({3}, {2}), ({6}, {2}), ({6}, {3}), ({3, 6}, {2}), ({2, 6}, {3}), ({6}, {2, 3})]


## Result for sale transaction data

The Data is a sale transaction data of 100000 transactions in numerical form as figure


In [18]:
def loadData() :
    datContent = [i.strip().split() for i in open("T10I4D100K.dat").readlines()]
    data = [  [int(y) for y in x] for x in datContent ]
    
    return data

In [20]:
data = loadData()
print("The first 2 transactions : ")
print(data[0:2])

The first 2 transactions : 
[[25, 52, 164, 240, 274, 328, 368, 448, 538, 561, 630, 687, 730, 775, 825, 834], [39, 120, 124, 205, 401, 581, 704, 814, 825, 834]]
